In [1]:
import pymysql
from math import *
import numpy as np

In [2]:
connection = pymysql.connect(host='127.0.0.1',
                             user='root',
                             password='5130309773',
                             db='ShanghaiOsm',
                             charset='utf8mb4',
                             port=3306,
                             cursorclass=pymysql.cursors.DictCursor)
cursor = connection.cursor()

In [3]:
cursor.execute("SELECT NodeID, Lat, Lon from Node")
Node2Pos = {i['NodeID']: (i['Lat'], i['Lon']) for i in cursor.fetchall()}

In [4]:
Way2Nodes = {}
cursor.execute("SELECT WayID, NodeID, OrderNum from WayNode")
for i in cursor.fetchall():
    if i['WayID'] not in Way2Nodes:
        Way2Nodes[i['WayID']] = []
    Way2Nodes[i['WayID']].append((i['OrderNum'], i['NodeID']))


In [5]:
for v in Way2Nodes.values():
    v.sort()

In [32]:
Node2Plane = {k: (pi - log(tan(pi / 4 + v[0]/180*pi / 2)), v[1]/180*pi + pi) for k, v in Node2Pos.items()}

In [7]:
Node2Pos[1]

(31.226864, 121.531082)

In [8]:
Way2Nodes[4531289]

[(0, 28111346),
 (1, 28111345),
 (2, 28111357),
 (3, 28111354),
 (4, 28111352),
 (5, 28111350),
 (6, 28111348),
 (7, 28111346)]

In [11]:
poss = np.array([i for i in Node2Pos.values()])

In [15]:
print(poss.max(axis=0))
print(poss.min(axis=0))

[  31.787997  121.972899]
[  30.614001  119.787003]


In [49]:
dist = {k:[] for k in Way2Nodes.keys()}
R = 6378137
lon_base = 95000
lat_base = 111122
diffs = []
for k, v in Way2Nodes.items():
    if k == 4531289:
        print([Node2Pos[i[1]] for i in v])
    for i in range(len(v) - 1):
        lat_diff = (Node2Pos[v[i][1]][0] - Node2Pos[v[i + 1][1]][0]) * lat_base
        lon_diff = (Node2Pos[v[i][1]][1] - Node2Pos[v[i + 1][1]][1]) * lon_base
        dist[k].append(sqrt(lat_diff * lat_diff + lon_diff * lon_diff))
        lat0 = Node2Pos[v[i][1]][0] / 180 * pi
        lat1 = Node2Pos[v[i + 1][1]][0] / 180 * pi
        lon0 = Node2Pos[v[i][1]][1] / 180 * pi
        lon1 = Node2Pos[v[i + 1][1]][1] / 180 * pi
        a = lat0 - lat1
        b = lon0 - lon1
        C = 2 * asin(sqrt(sin(a/2)**2+cos(lat0)*cos(lat1)*sin(b/2)**2))
        dist[k].append(C * R)
        diffs.append(abs(dist[k][-2] - dist[k][-1]))

[(30.96283, 120.10581), (30.96232, 120.10582), (30.96246, 120.10641), (30.96259, 120.10656), (30.96311, 120.10669), (30.96308, 120.1055), (30.96283, 120.10551), (30.96283, 120.10581)]


In [42]:
dist[4531289]

[56.68021545258717,
 56.7809646982009,
 58.28264717872491,
 58.43618118536394,
 20.312588981928037,
 20.357941072321125,
 59.09391943551646,
 59.20132395391415,
 113.33703822169507,
 113.64212171511934,
 27.79680708742228,
 27.846238673832435,
 28.560000000948094,
 28.636980603576397]

In [50]:
max(diffs)

199.38340325567697

In [45]:
max(diffs)

199.38340325567697

In [51]:
dist_list = []
for k, v in dist.items():
    if v != []:
        if max(v) > 10000:
            print(k, v)
    dist_list += v

364661234 [5197.320827314022, 5229.197285398483, 2873.078334676405, 2890.7812886960146, 4401.873020283575, 4428.618435940512, 494.2850000008292, 497.30945022204617, 747.4511044005092, 751.282710728887, 628.8358905434573, 631.7487235036068, 2584.4382959604254, 2590.8838807575216, 2579.6928072334504, 2594.104745918526, 1912.8082370405214, 1917.1712304723783, 744.8491180655658, 746.7133987720018, 26505.70624590004, 26553.37247660303, 15835.45549695986, 15869.776386723159, 8365.59297568774, 8381.107551383686, 7875.678992290201, 7888.43965932056, 7741.665778680329, 7750.804769780538, 13272.836365412053, 13281.5267179821]
27303455 [5726.167071695867, 5718.22234844412, 11234.04427301246, 11222.486226088467, 9666.318238717797, 9660.405646019864, 7140.1031193547215, 7139.974717193297, 8150.430711031138, 8151.697370891227, 4692.243406567533, 4693.089761804569, 8369.737445156026, 8371.217328301038]
319101439 [17905.083385933067, 17941.96075600684, 5086.948170957283, 5106.567923779954, 8302.651339

In [52]:
len(dist_list)

2344322

In [53]:
max(dist_list)

36725.28121639216

In [55]:
num = 0
for i in dist_list[::2]:
    num += i // 100
num

579744.0

In [95]:
R = 6378137
virtual_nodes = []
way_virtual_node = []
count = 1
for k, v in Way2Nodes.items():
    for i in range(len(v) - 1):
        lat0 = Node2Pos[v[i][1]][0] / 180 * pi
        lat1 = Node2Pos[v[i + 1][1]][0] / 180 * pi
        lon0 = Node2Pos[v[i][1]][1] / 180 * pi
        lon1 = Node2Pos[v[i + 1][1]][1] / 180 * pi
        a = lat0 - lat1
        b = lon0 - lon1
        dis = 2 * asin(sqrt(sin(a/2)**2+cos(lat0)*cos(lat1)*sin(b/2)**2)) * R
        if dis > 100:
            step_lat = (lat1 - lat0) / (dis // 100)
            step_lon = (lon1 - lon0) / (dis // 100)
            for j in range(int(dis // 100)):
                virtual_nodes.append((-count, Node2Pos[v[i][1]][0] + step_lat * (j + 1), Node2Pos[v[i][1]][1] + step_lon * (j + 1)))
                way_virtual_node.append((k, -count))
                count += 1

In [96]:
virtual_nodes = ['({0:d},{1:f},{1:f},POINT({1:f}, {1:f}),\'[]\')'.format(*i) for i in virtual_nodes]
way_virtual_node = ['(%d,%d,-1)' % i for i in way_virtual_node]
cursor.execute('LOCK TABLES Node WRITE, WayNode WRITE')
cursor.execute('/*!40000 ALTER TABLE `Node` DISABLE KEYS */')
cursor.execute('/*!40000 ALTER TABLE `Waynode` DISABLE KEYS */')

0

In [97]:
cursor.execute('delete from Node where NodeID < 0')
cursor.execute('delete from WayNode where NodeID < 0')

581360

In [99]:
for i in range(0, len(virtual_nodes), 10000):
    cursor.execute('INSERT into Node(NodeID,Lat,Lon,Pos,TagData) values %s' % ','.join(virtual_nodes[i:i+10000]))
    connection.commit()
for i in range(0, len(way_virtual_node), 10000):
    cursor.execute('INSERT into WayNode(WayID,NodeID,OrderNum) values %s' % ','.join(way_virtual_node[i:i+10000]))
    connection.commit()

In [100]:
cursor.execute('/*!40000 ALTER TABLE `Node` ENABLE KEYS */')
cursor.execute('/*!40000 ALTER TABLE `WayNode` ENABLE KEYS */')
cursor.execute('UNLOCK TABLES')

0

In [65]:
nodeids = [i for i in Node2Pos.keys()]

In [67]:
min(nodeids)

1

In [76]:
cursor.execute('delete from Node where NodeID < 0')

581360

In [87]:
cursor.execute('unlock tables')

0

In [84]:
a,b = [1,2]

In [98]:
virtual_nodes[0]

"(-1,31.525839,31.525839,POINT(31.525839, 31.525839),'[]')"